## Open Exploration

Explanation to what we did + table of contents

### Import Libraries

In [1]:
# General libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import copy

# Import problem formulation
from problem_formulation import get_model_for_problem_formulation

# EMA workbench imports
from ema_workbench import (Model, Policy, MultiprocessingEvaluator, ScalarOutcome, RealParameter, 
                           IntegerParameter, CategoricalParameter, optimize, Scenario, 
                           Constant, SequentialEvaluator)
#from ema_workbench.em_framework.evaluators import perform_experiments,LHS, SOBOL, MORRIS, SequentialEvaluator, BaseEvaluator
from ema_workbench.em_framework.samplers import sample_uncertainties, sample_levers
from ema_workbench.util import ema_logging, save_results, load_results

ema_logging.log_to_stderr(ema_logging.INFO)

<Logger EMA (DEBUG)>

## Worst Case Scenario 

In [2]:
# define the problem formulation between 0 and 9
# Problem formulation 6 is specific to Dike Ring 3
dike_model, planning_steps = get_model_for_problem_formulation(6)

In [3]:
# Set uncertainties and levers variables
uncertainties = copy.deepcopy(dike_model.uncertainties)
levers = copy.deepcopy(dike_model.levers)

In [4]:
# Create a reference policy where no action is taken
# Create a reference policy where no action is taken
policies = [Policy('null_policy', **{'0_RfR 0':0,'0_RfR 1':0,'0_RfR 2':0,
                                  '1_RfR 0':0,'1_RfR 1':0,'1_RfR 2':0,
                                  '2_RfR 0':0,'2_RfR 1':0,'2_RfR 2':0,
                                  '3_RfR 0':0,'3_RfR 1':0,'3_RfR 2':0,
                                  '4_RfR 0':0,'4_RfR 1':0,'4_RfR 2':0,
                                  'A.1_DikeIncrease 0':0,
                                  'A.2_DikeIncrease 0':0,
                                  'A.3_DikeIncrease 0':0,
                                  'A.4_DikeIncrease 0':0,
                                  'A.5_DikeIncrease 0':0,
                                  'A.1_DikeIncrease 1':0,
                                  'A.2_DikeIncrease 1':0,
                                  'A.3_DikeIncrease 1':0,
                                  'A.4_DikeIncrease 1':0,
                                  'A.5_DikeIncrease 1':0,
                                  'A.1_DikeIncrease 2':0,
                                  'A.2_DikeIncrease 2':0,
                                  'A.3_DikeIncrease 2':0,
                                  'A.4_DikeIncrease 2':0,
                                  'A.5_DikeIncrease 2':0,
                                  'EWS_DaysToThreat':0})
                                  ]

In [5]:
n_scenarios = 100

ema_logging.log_to_stderr(ema_logging.INFO)
 
with MultiprocessingEvaluator(dike_model) as evaluator:
    experiments, outcomes = evaluator.perform_experiments(scenarios=n_scenarios, policies=policies)

[MainProcess/INFO] pool started with 8 workers
[MainProcess/INFO] performing 100 scenarios * 1 policies * 1 model(s) = 100 experiments
100%|████████████████████████████████████████| 100/100 [00:16<00:00,  5.89it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [6]:
#Save the results
save_results([experiments, outcomes], 'results/10000Scenarios_NullPolicy_PF6.tar.gz')
# load the worst case results for further analysis 
experiments, outcomes = load_results('results/10000Scenarios_NullPolicy_PF6.tar.gz')

[MainProcess/INFO] results saved successfully to /Users/philipmuller/Documents/GitHub/model-based-decision-making/final assignment/results/10000Scenarios_NullPolicy_PF6.tar.gz
[MainProcess/INFO] results loaded successfully from /Users/philipmuller/Documents/GitHub/model-based-decision-making/final assignment/results/10000Scenarios_NullPolicy_PF6.tar.gz


## Here actual Open exploration shit


### show that random policy is better than no policy

In [7]:
# Create a policy where action is taken randomly
# compare the performance of that policy to the previously computed null_policy


import random # for randomly deciding to switch policy on/ off

random_policies = [Policy('alpha_random_policy', **{'3_RfR 0':random.randint(0,1),
                                  '3_RfR 1':random.randint(0,1),
                                  '3_RfR 2':random.randint(0,1),
                                  'A.1_DikeIncrease 0':random.randint(0,10),
                                  'A.2_DikeIncrease 0':random.randint(0,10),
                                  'A.3_DikeIncrease 0':random.randint(0,10),
                                  'A.4_DikeIncrease 0':random.randint(0,10),
                                  'A.5_DikeIncrease 0':random.randint(0,10),
                                  'A.1_DikeIncrease 1':random.randint(0,10),
                                  'A.2_DikeIncrease 1':random.randint(0,10),
                                  'A.3_DikeIncrease 1':random.randint(0,10),
                                  'A.4_DikeIncrease 1':random.randint(0,10),
                                  'A.5_DikeIncrease 1':random.randint(0,10),
                                  'A.1_DikeIncrease 2':random.randint(0,10),
                                  'A.2_DikeIncrease 2':random.randint(0,10),
                                  'A.3_DikeIncrease 2':random.randint(0,10),
                                  'A.4_DikeIncrease 2':random.randint(0,10),
                                  'A.5_DikeIncrease 2':random.randint(0,10),
                                  'EWS_DaysToThreat':random.randint(0,15)})]
'''
                Policy('beta_random_policy', **{'3_RfR 0':random.randint(0,1),
                                  '3_RfR 1':random.randint(0,1),
                                  '3_RfR 2':random.randint(0,1),
                                  'A.1_DikeIncrease 0':random.randint(0,10),
                                  'A.2_DikeIncrease 0':random.randint(0,10),
                                  'A.3_DikeIncrease 0':random.randint(0,10),
                                  'A.4_DikeIncrease 0':random.randint(0,10),
                                  'A.5_DikeIncrease 0':random.randint(0,10),
                                  'A.1_DikeIncrease 1':random.randint(0,10),
                                  'A.2_DikeIncrease 1':random.randint(0,10),
                                  'A.3_DikeIncrease 1':random.randint(0,10),
                                  'A.4_DikeIncrease 1':random.randint(0,10),
                                  'A.5_DikeIncrease 1':random.randint(0,10),
                                  'A.1_DikeIncrease 2':random.randint(0,10),
                                  'A.2_DikeIncrease 2':random.randint(0,10),
                                  'A.3_DikeIncrease 2':random.randint(0,10),
                                  'A.4_DikeIncrease 2':random.randint(0,10),
                                  'A.5_DikeIncrease 2':random.randint(0,10),
                                  'EWS_DaysToThreat':random.randint(0,15)}),
                Policy('gamma_random_policy', **{'3_RfR 0':random.randint(0,1),
                                  '3_RfR 1':random.randint(0,1),
                                  '3_RfR 2':random.randint(0,1),
                                  'A.1_DikeIncrease 0':random.randint(0,10),
                                  'A.2_DikeIncrease 0':random.randint(0,10),
                                  'A.3_DikeIncrease 0':random.randint(0,10),
                                  'A.4_DikeIncrease 0':random.randint(0,10),
                                  'A.5_DikeIncrease 0':random.randint(0,10),
                                  'A.1_DikeIncrease 1':random.randint(0,10),
                                  'A.2_DikeIncrease 1':random.randint(0,10),
                                  'A.3_DikeIncrease 1':random.randint(0,10),
                                  'A.4_DikeIncrease 1':random.randint(0,10),
                                  'A.5_DikeIncrease 1':random.randint(0,10),
                                  'A.1_DikeIncrease 2':random.randint(0,10),
                                  'A.2_DikeIncrease 2':random.randint(0,10),
                                  'A.3_DikeIncrease 2':random.randint(0,10),
                                  'A.4_DikeIncrease 2':random.randint(0,10),
                                  'A.5_DikeIncrease 2':random.randint(0,10),
                                  'EWS_DaysToThreat':random.randint(0,15)})                  
                                  ]'''

"\n                Policy('beta_random_policy', **{'3_RfR 0':random.randint(0,1),\n                                  '3_RfR 1':random.randint(0,1),\n                                  '3_RfR 2':random.randint(0,1),\n                                  'A.1_DikeIncrease 0':random.randint(0,10),\n                                  'A.2_DikeIncrease 0':random.randint(0,10),\n                                  'A.3_DikeIncrease 0':random.randint(0,10),\n                                  'A.4_DikeIncrease 0':random.randint(0,10),\n                                  'A.5_DikeIncrease 0':random.randint(0,10),\n                                  'A.1_DikeIncrease 1':random.randint(0,10),\n                                  'A.2_DikeIncrease 1':random.randint(0,10),\n                                  'A.3_DikeIncrease 1':random.randint(0,10),\n                                  'A.4_DikeIncrease 1':random.randint(0,10),\n                                  'A.5_DikeIncrease 1':random.randint(0,10),\n     

In [8]:
'''n_scenarios = 100

ema_logging.log_to_stderr(ema_logging.INFO)
 
with MultiprocessingEvaluator(dike_model) as evaluator:
    experiments, outcomes = evaluator.perform_experiments(scenarios=n_scenarios, policies=random_policies)'''

'n_scenarios = 100\n\nema_logging.log_to_stderr(ema_logging.INFO)\n \nwith MultiprocessingEvaluator(dike_model) as evaluator:\n    experiments, outcomes = evaluator.perform_experiments(scenarios=n_scenarios, policies=random_policies)'

In [36]:
n_scenarios = 100

ema_logging.log_to_stderr(ema_logging.INFO)

# Model Imports
from dike_model_function import DikeNetwork
from problem_formulation import get_model_for_problem_formulation

dike_model, time_step = get_model_for_problem_formulation(6)
 
with MultiprocessingEvaluator(dike_model) as evaluator:
    random_experiments, random_outcomes = evaluator.perform_experiments(scenarios=n_scenarios, policies=random_policies)

[MainProcess/INFO] pool started with 8 workers
[MainProcess/INFO] performing 100 scenarios * 1 policies * 1 model(s) = 100 experiments
100%|████████████████████████████████████████| 100/100 [00:14<00:00,  6.85it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [37]:
#Save the results
save_results([random_experiments, random_outcomes], 'results/10000Scenarios_RandomPolicy_PF6.tar.gz')
# load the worst case results for further analysis 
random_experiments, random_outcomes = load_results('results/10000Scenarios_RandomPolicy_PF6.tar.gz')


random_outcomes = pd.DataFrame(random_outcomes).drop(columns=['A.1_Dike Investment Costs',
                                                              'A.2_Dike Investment Costs',
                                                              'A.3_Dike Investment Costs',
                                                              'A.4_Dike Investment Costs',
                                                              'A.5_Dike Investment Costs',
                                                              'RfR Total Costs',
                                                              'Expected Evacuation Costs'])

#put outcomes from null_policy into similar format
outcomes = pd.DataFrame(outcomes).drop(columns=['A.1_Dike Investment Costs',
                                                              'A.2_Dike Investment Costs',
                                                              'A.3_Dike Investment Costs',
                                                              'A.4_Dike Investment Costs',
                                                              'A.5_Dike Investment Costs',
                                                              'RfR Total Costs',
                                                              'Expected Evacuation Costs'])

[MainProcess/INFO] results saved successfully to /Users/philipmuller/Documents/GitHub/model-based-decision-making/final assignment/results/10000Scenarios_RandomPolicy_PF6.tar.gz
[MainProcess/INFO] results loaded successfully from /Users/philipmuller/Documents/GitHub/model-based-decision-making/final assignment/results/10000Scenarios_RandomPolicy_PF6.tar.gz


KeyError: "['A.1_Dike Investment Costs', 'A.2_Dike Investment Costs', 'A.3_Dike Investment Costs', 'A.4_Dike Investment Costs', 'A.5_Dike Investment Costs', 'RfR Total Costs', 'Expected Evacuation Costs'] not found in axis"

In [27]:
'''
options: 

1. do sample boxplots 
2. do a heatmap of p_values
'''

Index(['A.1_Expected Annual Damage', 'A.1_Expected Number of Deaths',
       'A.2_Expected Annual Damage', 'A.2_Expected Number of Deaths',
       'A.3_Expected Annual Damage', 'A.3_Expected Number of Deaths',
       'A.4_Expected Annual Damage', 'A.4_Expected Number of Deaths',
       'A.5_Expected Annual Damage', 'A.5_Expected Number of Deaths'],
      dtype='object')

In [ ]:
# option 1: boxplots


import seaborn as sns
import matplotlib.pyplot as plt

# Example data
data1 = [1, 2, 3, 4, 5]
data2 = [3, 4, 5, 6, 7]

# Create a figure and axes
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

# Create the first boxplot on the left
sns.boxplot(data=data1, ax=axes[0])
axes[0].set_title('Boxplot 1')

# Create the second boxplot on the right
sns.boxplot(data=data2, ax=axes[1])
axes[1].set_title('Boxplot 2')

# Set the spacing between the subplots
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
# option 2: statistical tests and heatmap of p_values

#random_experiments, random_outcomes = pd.DataFrame(random_experiments), pd.DataFrame(random_outcomes)

from scipy.stats import normaltest, mannwhitneyu, ttest_ind

for col in outcomes.columns: 
    #statistic, p_value = normaltest(outcomes[col])
    print(outcomes.columns)
    #print(f'normaltest for {col} and: p-value = {p_value}')
    

for col in random_outcomes.columns:
    statistic, p_value = normaltest(random_outcomes[col])
    print(f'normaltest for {col} and: p-value = {p_value}')

RangeIndex(start=0, stop=100, step=1)
RangeIndex(start=0, stop=100, step=1)
RangeIndex(start=0, stop=100, step=1)
RangeIndex(start=0, stop=100, step=1)
RangeIndex(start=0, stop=100, step=1)
RangeIndex(start=0, stop=100, step=1)
RangeIndex(start=0, stop=100, step=1)
RangeIndex(start=0, stop=100, step=1)
RangeIndex(start=0, stop=100, step=1)
RangeIndex(start=0, stop=100, step=1)
RangeIndex(start=0, stop=100, step=1)
RangeIndex(start=0, stop=100, step=1)
RangeIndex(start=0, stop=100, step=1)
RangeIndex(start=0, stop=100, step=1)
RangeIndex(start=0, stop=100, step=1)
RangeIndex(start=0, stop=100, step=1)
RangeIndex(start=0, stop=100, step=1)
RangeIndex(start=0, stop=100, step=1)
RangeIndex(start=0, stop=100, step=1)
RangeIndex(start=0, stop=100, step=1)
RangeIndex(start=0, stop=100, step=1)
RangeIndex(start=0, stop=100, step=1)
RangeIndex(start=0, stop=100, step=1)
RangeIndex(start=0, stop=100, step=1)
RangeIndex(start=0, stop=100, step=1)
RangeIndex(start=0, stop=100, step=1)
RangeIndex(s

In [ ]:
outcomes.columns

RangeIndex(start=0, stop=100, step=1)

In [ ]:
for col_random in random_outcomes.index:
    for col_outcomes in outcomes.index:
        statistic, p_value = ttest_ind(random_outcomes[col_random], outcomes[col_outcomes])
        print(f"T-test between {col_random} and {col_outcomes}: p-value = {p_value}")


# repeat for manwhitneyu


KeyError: 'Expected Annual Damage'

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Example data
data1 = [1, 2, 3, 4, 5]
data2 = [3, 4, 5, 6, 7]

# Create a figure and axes
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

# Create the first boxplot on the left
sns.boxplot(data=data1, ax=axes[0])
axes[0].set_title('Boxplot 1')

# Create the second boxplot on the right
sns.boxplot(data=data2, ax=axes[1])
axes[1].set_title('Boxplot 2')

# Set the spacing between the subplots
plt.tight_layout()

# Show the plot
plt.show()
